In [ ]:
# importing sys
import sys

# adding Folder_2 to the system path
sys.path.insert(0, '/home/jeongeun/test_env/Open-Set-Object-Detection')

In [ ]:
# AI2 THOR
# !pkill thor_*
import ai2thor
from ai2thor.controller import Controller,BFSController
from ai2thor.platform import CloudRendering
from ithor_tools.vis_tool import *
from ithor_tools.transform import cornerpoint_projection,depth2world
from ithor_tools.map2 import single_scenemap
from ithor_tools.landmark_utils import gather,gather2,vis_panorama,Word_Dict,choose_ladmark, landmark_names
from eval_ithor.objects import choose_query_objects,detect

from co_occurance.comet_co import co_occurance_score
import random
import math
# from IPython.display import display
# from moviepy.editor import ImageSequenceClip,VideoFileClip


# Planning Module
from RRT import gridmaprrt as rrt
from RRT import gridmaprrt_pathsmoothing as smoothing

from FBE.memory import fbe_map
from FBE.schedular import traj_schedular

In [ ]:
# Detector module
from data.phase_1 import load_voc_instances,VOC_CLASS_NAMES
import torch
import cv2
import copy
import matplotlib.pyplot as plt
from structures.box import Boxes
from engine.predictor import DefaultPredictor

from  config.config import get_cfg
from model.rcnn import GeneralizedRCNN
from detector.postprocess import postprocess,plot_openset,plot_candidate

## Matching Module
from detector.query_matching import matcher

In [ ]:
'''
config file
'''

print(torch.cuda.device_count())
torch.cuda.set_device(0)
print(torch.cuda.current_device())
cfg = get_cfg()
cfg.merge_from_file('../Open-Set-Object-Detection/config_files/voc.yaml')
cfg.MODEL.SAVE_IDX=19 #22
cfg.MODEL.RPN.USE_MDN=False
cfg.log = False 
cfg.MODEL.ROI_HEADS.USE_MLN = True
cfg.MODEL.ROI_HEADS.AUTO_LABEL = False
cfg.MODEL.ROI_HEADS.AF = 'baseline'
cfg.MODEL.RPN.AUTO_LABEL = False
cfg.MODEL.ROI_BOX_HEAD.USE_FD = False
cfg.MODEL.RPN.AUTO_LABEL_TYPE = 'sum'
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 21
cfg.INPUT.RANDOM_FLIP = "none"
cfg.MODEL.ROI_HEADS.UNCT = True
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.2
cfg.PATH = '../Open-Set-Object-Detection'
cfg.phase = 'voc'
# cfg.merge_from_list(args.opts)
RPN_NAME = 'mdn' if cfg.MODEL.RPN.USE_MDN else 'base'
ROI_NAME = 'mln' if cfg.MODEL.ROI_HEADS.USE_MLN else 'base'
MODEL_NAME = RPN_NAME + ROI_NAME
# cfg.merge_from_list(args.opts)
cfg.freeze()
# wandb.init(config=cfg,tags= 'temp',name = 'temp',project='temp')

In [ ]:
device = 'cuda:1'
model = GeneralizedRCNN(cfg,device = device).to(device)
state_dict = torch.load('../Open-Set-Object-Detection/ckpt/{}/{}_{}_15000.pt'.format(cfg.MODEL.ROI_HEADS.AF,cfg.MODEL.SAVE_IDX,MODEL_NAME),map_location=device)
pretrained_dict = {k: v for k, v in state_dict.items() if k in model.state_dict()}
model.load_state_dict(pretrained_dict)

predictor = DefaultPredictor(cfg,model)

In [ ]:
gridSize=0.05
scene_name = "FloorPlan_Val1_4"
controller = Controller(
    agentMode="locobot",
    platform=CloudRendering,
    visibilityDistance=1.5,
    scene = scene_name,
    gridSize=gridSize,
    movementGaussianSigma=0,
    rotateStepDegrees=90,
    rotateGaussianSigma=0,
    renderClassImage = True,
    renderDepthImage=False,
    renderInstanceSegmentation=False,
    width=300,
    height=300,
    fieldOfView=60
)

In [ ]:
controller.reset(
    # makes the images a bit higher quality
    width=800,
    height=800,

    # Renders several new image modalities
    renderDepthImage=True,
    renderClassImage = True,
    renderSemanticSegmentation=False,
    renderNormalsImage=False
)
scene_bounds = controller.last_event.metadata['sceneBounds']['center']
controller.step(
    action="AddThirdPartyCamera",
    position=dict(x=scene_bounds['x'], y=5.0, z=scene_bounds['z']),
    rotation=dict(x=90, y=0, z=0),
    orthographic=True,
    orthographicSize= 5.0, fieldOfView=100,
    skyboxColor="white"
)
controller.step(dict(action='GetReachablePositions'))
rstate = controller.last_event.metadata['actionReturn']

controller.step(
    action="Teleport",
    position = rstate[100],
    rotation = dict(x=0,y=0,z=0)
)

pos = controller.last_event.metadata['agent']['position']
pos = [pos['x'],pos['z']]
objects = controller.last_event.metadata['objects']

In [ ]:
df = show_objects_table(objects)
df

In [ ]:
query_object = objects[13]

In [ ]:
landmark_cat = [Word_Dict[l] for l in landmark_names]
co_occurance_scoring = co_occurance_score('cuda:0')
co_occurance_scoring.landmark_init(landmark_cat)


In [ ]:
co_occurance = co_occurance_scoring.score(query_object['objectType'])

In [ ]:
scene_bounds = controller.last_event.metadata['sceneBounds']['cornerPoints']
scene_bounds = cornerpoint_projection(scene_bounds)


In [ ]:
scene_memory = fbe_map(scene_bounds,rstate,landmark_names=landmark_names, stepsize=0.1)
sche = traj_schedular(landmark_names,controller)
sche.set_score(co_occurance)

In [ ]:
VOC_CLASS_NAMES = (
    "aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat",
    "chair", "cow", "diningtable", "dog", "horse", "motorbike", "person",
    "pottedplant", "sheep", "sofa", "train", "tvmonitor", 'unknown'
)

In [ ]:
visible_landmark_names = ['Bed', 'DiningTable', 'StoveBurner', 'Desk','Drawer','Television','Sofa','SideTable','CoffeeTable','ArmChair',]

In [ ]:
clip_matcher = matcher(visible_landmark_names,device='cuda:0')
clip_matcher.tokenize(query_object['objectType'])


In [ ]:
controller.step(
    action="Teleport",
    position = rstate[100],
    rotation = dict(x=0,y=0,z=0)
)

In [ ]:
from eval_ithor.reset import get_min_dis

min_dis = get_min_dis(query_object,controller,None,None)
print(min_dis)

In [ ]:
rrtplanner = rrt.RRT(controller = controller, expand_dis=0.1,max_iter=10000,goal_sample_rate=20)
d2w = depth2world()

In [ ]:
total_patch = np.zeros((0,256,256,3),dtype=np.uint8)
total_mappoints = []
total_success = 0
total_path_len = 0

frames,single_pos,gt_boxes,gt_vis,detected_landmarks = gather2(controller,[query_object['objectId']],
                        predictor,postprocess,clip_matcher,
        visible_landmark_names,scene_memory,show=True)
print('gt_vis?',gt_vis)
candidate_patches, candidate_map_points,sucesses = detect(frames,single_pos,gt_boxes,controller,predictor,clip_matcher,d2w)

total_patch = np.concatenate((total_patch,candidate_patches),axis=0)
total_mappoints += candidate_map_points
total_success += sucesses
vis_panorama(frames,res=360)
scene_memory.visited(controller)
landmark_config = dict(name=landmark_names,color = scene_memory.landmark_colors)
cpos = controller.last_event.metadata['agent']['position']
candidate_trajs = []
for l in detected_landmarks:
    res = scene_memory.get_reachable(cpos,l)
    if not res[0] == None:
        res = dict(name=l,pos=res[0],rot = res[1])
        candidate_trajs.append(res)
waypoints = scene_memory.frontier_detection(cpos)
candidate_trajs += waypoints


cpos = controller.last_event.metadata['agent']['position']
imshow_grid = scene_memory.plot(controller.last_event.metadata['agent']['position'],candidate_trajs,query_object['position'])
plot_frames2(controller.last_event,imshow_grid,landmark_config)     

In [ ]:
traj_image = controller.last_event.third_party_camera_frames[0]

In [ ]:

NUM_WAYPOINT = 0
def draw_path(rrt,traj_image,path):
    traj_image = np.ascontiguousarray(traj_image, dtype=np.uint8)
    for idx in range(len(path)-1):
        prev_xz = rrt.rstate[path[idx]]
        next_xz = rrt.rstate[path[idx+1]]
        prev_w,prev_h = rrt.play_area.xz2coor(prev_xz[0],prev_xz[1])
        next_w,next_h = rrt.play_area.xz2coor(next_xz[0],next_xz[1])
        cv2.line(traj_image, (int(prev_w),int(prev_h)), (int(next_w),int(next_h)), (0,255,255), 2)
    # plt.imshow(traj_image)
    # plt.plot(path_wh[:,0], path_wh[:,1],'-c')

    # Cast start&end point coordinate in image
    # startx, startz = (rrt.start.x), (rrt.start.z)
    endx,   endz   = (rrt.end.x), (rrt.end.z)
    
    # startx, startz = rrt.play_area.xz2coor(startx, startz )
    endx,   endz   = rrt.play_area.xz2coor(endx,   endz   )
    
    # plt.plot(startx, startz,"xr")
    cv2.circle(traj_image, (int(endx),int(endz)), 5, (255,0,255), thickness=5)
    cv2.putText(traj_image, str(NUM_WAYPOINT), (int(endx),int(endz)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
    # plt.plot(endx, endz, "xr")
    plt.imshow(traj_image)
    plt.show()
    return traj_image


In [ ]:
angle = 60
step = 3
while not total_success:
    cpos = controller.last_event.metadata['agent']['position']
    path = sche.schedule(cpos,0,candidate_trajs)
    print(path)
    for p in path[1:]:
        if p == None:
            scene_memory.reset_gridmap()
            print('expolaration done')
            break
        # print(p)
        pos = controller.last_event.metadata['agent']['position']
        rrtplanner.set_start(pos)
        rrtplanner.set_goal(p['pos'])
        print("start planning")
        local_path = rrtplanner.planning(animation=False)
        try:
            smoothpath = smoothing.path_smoothing(rrtplanner,40,verbose=False)
        except:
            smoothpath = local_path
        to_pos = rrtplanner.rstate[smoothpath[-1]]
        # rrtplanner.plot_path(smoothpath)
        NUM_WAYPOINT +=1
        traj_image = draw_path(rrtplanner,traj_image,smoothpath)
        print("end planning")
        flag,path_len,frames = rrtplanner.go_with_teleport(smoothpath,maxspeed=0.2)
        print("end move")
        total_path_len += path_len
        if p['name'] == 'frontier':
            # controller.step(
            # action="Teleport",
            # position = dict(x=to_pos[0],y=0.91,z=to_pos[1]), rotation = dict(x=0,y=0,z=0)
            #     )
            # print(controller.last_event.metadata['lastActionSuccess'])
            scene_memory.reset_landmark(detected_landmarks)
            frames,single_pos,gt_boxes,gt_vis,detected_landmarks = gather2(controller,[query_object['objectId']],
                        predictor,postprocess,clip_matcher,
                    visible_landmark_names,scene_memory,show=True)
            print('gt_vis?',gt_vis)
            candidate_patches, candidate_map_points,sucesses = detect(frames,single_pos,gt_boxes,controller,predictor,clip_matcher,d2w)
            total_patch = np.concatenate((total_patch,candidate_patches),axis=0)
            total_mappoints += candidate_map_points
            total_success += sucesses
            vis_panorama(frames,res=360)

            scene_memory.visited(controller)
            landmark_config = dict(name=landmark_names,color = scene_memory.landmark_colors)
            cpos = controller.last_event.metadata['agent']['position']
            candidate_trajs = []
            for l in detected_landmarks:
                res = scene_memory.get_reachable(cpos,l)
                if not res[0] == None:
                    res = dict(name=l,pos=res[0],rot = res[1])
                    candidate_trajs.append(res)
            waypoints = scene_memory.frontier_detection(cpos)
            candidate_trajs += waypoints
        else:
            pos = controller.last_event.metadata['agent']['position']
            controller.step(
                action="Teleport",
                position = pos, rotation = dict(x=0,y=p['rot']-angle/2,z=0)
                    )
            print(controller.last_event.metadata['lastActionSuccess'])
            imshow_grid = scene_memory.plot(controller.last_event.metadata['agent']['position'],candidate_trajs,query_object['position'])
            plot_frames2(controller.last_event,imshow_grid,landmark_config)
            frames, single_pos,gt_boxes,gt_vis = gather(controller,[query_object['objectId']],step=step,angle=angle)
            print('gt_vis?',gt_vis)
            candidate_patches, candidate_map_points,sucesses = detect(frames,single_pos,gt_boxes,controller,predictor,clip_matcher,d2w)
            total_patch = np.concatenate((total_patch,candidate_patches),axis=0)
            total_mappoints += candidate_map_points
            total_success += sucesses
            vis_panorama(frames,res=angle)
            
        cpos = controller.last_event.metadata['agent']['position']
        imshow_grid = scene_memory.plot(controller.last_event.metadata['agent']['position'],candidate_trajs,query_object['position'])
        plot_frames2(controller.last_event,imshow_grid,landmark_config)  
        if total_success:
            break      


In [ ]:
img = controller.last_event.cv2img
pred = predictor(img)
pred_boxes, pred_classes,_ = postprocess(pred)
plot_openset(img,pred_boxes,pred_classes,visible_landmark_names)

In [ ]:
cpos = controller.last_event.metadata['agent']['position']
imshow_grid = scene_memory.plot(controller.last_event.metadata['agent']['position'],candidate_trajs,query_object['position'])
plot_frames2(controller.last_event,imshow_grid,landmark_config)  

print("Sucess?",total_success>0)
print("Total Path Length", total_path_len)
SPL = (total_success>0)*min_dis/total_path_len
print("SPL:",SPL )
plot_candidate(total_patch,total_mappoints,query_object['objectType'],scene_memory)

In [ ]:
# controller.stop_unity()

In [ ]:
size = len(total_patch)
plt.figure(figsize=(3*size,5))
plt.suptitle("Candidate Image of [{}]".format(query_object['objectType']))
for e, (patch,point) in enumerate(zip(total_patch,total_mappoints)):
    map = scene_memory.grid_map.copy()
    new_grid = scene_memory.xyz2grid(point)
    try:
        map[new_grid[0],new_grid[1],:] = [1,0.5,1]
    except:
        pass
    plt.subplot(2,size,e+1)
    plt.imshow(patch)
    plt.axis('off')

    plt.subplot(2,size,e+size+1)
    map = np.rot90(map)
    plt.imshow(map)
    plt.axis('off')
# plt.savefig('temp.png')
plt.show()

In [ ]:
controller.stop_unity()